In [1]:
!pip install datasets
!pip install tqdm
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.3/760.3 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64

In [13]:
import os
import shutil

# sample_data 폴더의 경로
sample_data_path = '/content/sample_data'

# sample_data 폴더가 존재하는지 확인
if os.path.exists(sample_data_path):
    # 폴더와 그 안의 모든 파일 및 디렉토리 삭제
    shutil.rmtree(sample_data_path)
    print('sample_data 폴더가 삭제되었습니다.')
else:
    print('sample_data 폴더가 존재하지 않습니다.')

sample_data 폴더가 삭제되었습니다.


In [3]:
import os
print('현재 작업 경로 :', os.getcwd())

현재 작업 경로 : /content


In [4]:
os.makedirs('./data/train/images')
os.makedirs('./data/train/labels')
os.makedirs('./data/validation/images')
os.makedirs('./data/validation/labels')

In [5]:
def get_index(item, list):
    if item in list:
        return list.index(item)

# box[0], box[1], box[2], box[3] : xmin, ymin, xmax, ymax
# 좌표변환 (xyxy -> xywhn)
def convert_box(box):
          dw, dh = 1. / 1024, 1. / 576
          x, y, w, h = (box[0] + box[2]) / 2.0 - 1, (box[1] + box[3]) / 2.0 - 1, box[2] - box[0], box[3] - box[1]
          return[x * dw, y * dh, w * dw, h * dh]

# 소수 6자리로 변환
def format_to_6_decimal_places(number_list):
    output_list = []
    for number in number_list:
        formatted_number = round(number, 6)
        output_list.append(str(formatted_number))
    return output_list


In [6]:
import huggingface_hub
huggingface_hub.login()

In [7]:
from datasets import load_dataset
ds = load_dataset("Hongik-ML-2024/processed-dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/31 [00:00<?, ?it/s]

In [8]:
import json

# 미리 생성해둔 object_list.json 작업 경로 내 로드 (After make_object_list.py run)
with open('object_list.json', "r") as f:
    object_list = json.load(f)

In [9]:
# train data 전처리
from tqdm import tqdm

train = ds['train']
train_data_dir = './data/train'
index = 0

for train_data in tqdm(train):
    # image 저장
    binary_data = train_data['image']
    with open(train_data_dir + f'/images/{index}.jpg', 'wb') as file:
        file.write(binary_data)

    # label text 생성 후 저장
    label_text = ''
    label = train_data['label']
    Annotations = label['Learning_Data_Info.']['Annotations']
    for annotation in Annotations:
        # 사물 index 알아내기
        object_num = get_index(annotation['Class_ID'], object_list)
        object_num = str(object_num)

        # 좌표 변환 (xyxy -> xywhn)
        box = annotation['Type_value']
        box = convert_box(box)
        box = format_to_6_decimal_places(box)
        coordinate = ' '.join(box)

        # 한줄 추가
        text = object_num + ' ' + coordinate
        label_text += text + '\n'
    label_text = label_text[:-1]

    with open(train_data_dir + f'/labels/{index}.txt', "w") as file:
        file.write(label_text)
    index += 1

100%|██████████| 586632/586632 [03:31<00:00, 2773.86it/s]


In [10]:
# validation data 전처리
val = ds['validation']
val_data_dir = './data/validation'

index = 0
for val_data in tqdm(val):
    # image 저장
    binary_data = val_data['image']
    with open(val_data_dir + f'/images/{index}.jpg', 'wb') as file:
        file.write(binary_data)

    # label text 생성 후 저장
    label_text = ''
    label = val_data['label']
    Annotations = label['Learning_Data_Info.']['Annotations']
    for annotation in Annotations:
        # 사물 index 알아내기
        object_num = get_index(annotation['Class_ID'], object_list)
        object_num = str(object_num)

        # 좌표 변환 (xyxy -> xywhn)
        box = annotation['Type_value']
        box = convert_box(box)
        box = format_to_6_decimal_places(box)
        coordinate = ' '.join(box)

        # 한줄 추가
        text = object_num + ' ' + coordinate
        label_text += text + '\n'
    label_text = label_text[:-1]

    with open(val_data_dir + f'/labels/{index}.txt', "w") as file:
        file.write(label_text)
    index += 1

100%|██████████| 73379/73379 [00:26<00:00, 2781.03it/s]


In [14]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(
   data='data/data.yaml',
   imgsz=1024,
   epochs=1,
   batch=8,
   name='yolov8n_custom'
)

Ultralytics YOLOv8.2.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data/data.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /content/data/train/labels.cache... 586632 images, 0 backgrounds, 0 corrupt: 100%|██████████| 586632/586632 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/data/validation/labels.cache... 73379 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73379/73379 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov8n_custom/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000182, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs/detect/yolov8n_custom
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      15.5G     0.9192      1.968      1.301         36       1024: 100%|██████████| 73329/73329 [2:23:47<00:00,  8.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/4587 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4587/4587 [07:46<00:00,  9.83it/s]


                   all      73379      93309      0.863      0.727       0.81       0.66

1 epochs completed in 2.530 hours.
Optimizer stripped from runs/detect/yolov8n_custom/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/yolov8n_custom/weights/best.pt, 6.3MB

Validating runs/detect/yolov8n_custom/weights/best.pt...
Ultralytics YOLOv8.2.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3015593 parameters, 0 gradients, 8.1 GFLOPs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4587/4587 [06:13<00:00, 12.28it/s]


                   all      73379      93309      0.862      0.728      0.809       0.66
                  Ball      73379        600      0.909      0.583      0.785      0.559
          Baseball Bat      73379        724      0.859      0.883      0.925      0.756
        Baseball Glove      73379        672      0.911      0.841      0.903      0.781
                 Bench      73379       4777      0.887      0.815      0.891      0.702
               Bicycle      73379       6859      0.937      0.903      0.961      0.793
                  Boat      73379        225      0.822      0.587      0.704      0.432
                  Book      73379       1925      0.876      0.714      0.821      0.735
                Bottle      73379       1730      0.933      0.795      0.879      0.708
                   Bus      73379        376      0.796      0.604      0.709      0.557
              Calendar      73379       1290      0.907      0.922      0.957      0.885
                Camer

In [16]:
# 학습 결과 드라이브에 저장
from google.colab import drive
drive.mount('/content/drive')

# 원본 경로
source_path = '/content/runs/detect/yolov8n_custom'

# 목적지 경로
destination_path = '/content/drive/MyDrive/ML_project/yolov8n_custom'
# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# '/content' 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
